In [1]:
import pandas as pd
import numpy as np
import cv2
from pathlib import Path
from src.tracking.position_calculation import project_3d_to_2d_left

In [2]:
root_dir = Path.cwd().parent
data_dir = root_dir / "data"

csv_file = data_dir / "test.csv"

In [3]:
print(csv_file)

/Users/juan/Documents/DTU/2 Semester MSc/Perception for AS/Project/PFAS---final-project/data/test.csv


In [4]:
object_columns = [
    "frame",
    "object_id",
    "object_type",
    "observed",
    "position_x",
    "position_y",
    "position_z",
    "last_seen",
    "bbox_cx",
    "bbox_cy",
    "pred_bbox_cx",
    "pred_bbox_cy",
    "bbox_w",
    "bbox_h",
]



In [5]:
#pandas read csv, sep = ",", columns = object_columns, header is first row

df = pd.read_csv(csv_file, sep = ",", names = object_columns, header = 0)
df.head()

,frame,object_id,object_type,observed,position_x,position_y,position_z,last_seen,bbox_cx,bbox_cy,pred_bbox_cx,pred_bbox_cy,bbox_w,bbox_h
0,0,998ba190-70a8-4abd-ba98-55a55d8f2e92,Car,True,8.634795,-0.458963,19.732910,0,149.327881,280.018646,149.32788,280.01865,297.531616,178.197372
1,0,3bc56b0c-84f6-4884-b79a-c479ff73c214,Car,True,12.963955,4.762627,71.904917,0,465.501831,182.961258,465.50183,182.96126,37.000275,24.454758
2,0,f15838b1-7baa-47bb-b6cb-d7f42c59309f,Pedestrian,True,-3.297716,0.492568,14.615819,0,964.320068,218.466339,964.32007,218.46634,55.109314,149.362869
3,0,dcc8e6d8-9738-404c-a2b1-0be3dd273803,Pedestrian,True,0.926048,0.215623,11.055467,0,584.122681,231.438232,584.12270,231.43823,64.892944,163.676086
4,0,a66e31a2-a545-49d4-b58d-4fdbe247b486,Pedestrian,True,14.112621,0.406672,31.187092,0,130.979370,227.539825,130.97937,227.53983,48.637802,143.869690


In [6]:
# get all unique object ids
object_ids = df["object_id"].unique()

# generate a random color for each object id
colors = np.random.randint(0, 255, size=(len(object_ids), 3), dtype=np.uint8)

# create a dictionary of object ids and colors
color_dict = dict(zip(object_ids, colors))

In [7]:
color_dict["f15838b1-7baa-47bb-b6cb-d7f42c59309f"]

array([137, 184,  45], dtype=uint8)

In [8]:
dp_left = data_dir / "raw"/"final_project_2023_rect" / "seq_03" / "image_02/data"

glob_left = sorted(dp_left.glob("*.png"))


In [9]:
print(glob_left[0])

/Users/juan/Documents/DTU/2 Semester MSc/Perception for AS/Project/PFAS---final-project/data/raw/final_project_2023_rect/seq_03/image_02/data/0000000000.png


In [20]:
data_dir / "results" 
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
video_writer = cv2.VideoWriter(f"{data_dir}/results/frame_0.jpg" , fourcc, 10, (1224,370))

[ERROR:0@209.252] global cap.cpp:597 open VIDEOIO(CV_IMAGES): raised OpenCV exception:

OpenCV(4.7.0) /Users/runner/work/opencv-python/opencv-python/opencv/modules/videoio/src/cap_images.cpp:253: error: (-5:Bad argument) CAP_IMAGES: can't find starting number (in the name of file): ../data/results in function 'icvExtractPattern'




In [19]:
# for each image in the left image directory draw the bounding boxes with the colour of the object_id of the objects in the image and save the image
for frame, img_left_path in enumerate(glob_left):
    print(f"Processing frame {frame}: {img_left_path.name}")

    # read images
    img_left = cv2.imread(str(img_left_path))

    # get the rows of the dataframe for the current frame
    df_frame = df[df["frame"] == frame]

    

    # for each row in the dataframe draw the bounding box with the colour of the object_id
    for index, row in df_frame.iterrows():

        # if the absolute difference between last seen frame and frame is larger than 10, skip
        if abs(row["last_seen"] - frame) > 35:
            continue

        object_id = row["object_id"]
        bbox_cx = row["bbox_cx"]
        bbox_cy = row["bbox_cy"]
        bbox_w = row["bbox_w"]
        bbox_h = row["bbox_h"]

        pred_cx = row["pred_bbox_cx"]
        pred_cy = row["pred_bbox_cy"]
        # pred_z = row["prediciton_z"]

        # if pred_z < 0.0001:       
        #     pred_z = 0.0001 

        # pred_ul, pred_vl = project_3d_to_2d_left([pred_x, pred_y, pred_z])

        

        # get the colour for the object_id
        color = tuple(color_dict[object_id])

        # only draw bounding box if observed
        # if row["observed"]:
        # draw the bounding box
        cv2.rectangle(
            img_left,
            (int(bbox_cx - bbox_w / 2), int(bbox_cy - bbox_h / 2)),
            (int(bbox_cx + bbox_w / 2), int(bbox_cy + bbox_h / 2)),
            (int(color[0]), int(color[1]), int(color[2])),
            2,
        )

        # reduce opacity of the bounding box
        overlay = img_left.copy()
        cv2.rectangle(
            overlay,
            (int(pred_cx - bbox_w / 2), int(pred_cy - bbox_h / 2)),
            (int(pred_cx + bbox_w / 2), int(pred_cy + bbox_h / 2)),
            (int(color[0]), int(color[1]), int(color[2])),
            2,
        )

        # draw the prediction as a cross
        cv2.line(
            img_left,
            (int(pred_cx - 10), int(pred_cy)),
            (int(pred_cx + 10), int(pred_cy)),
            (int(color[0]), int(color[1]), int(color[2])),
            2,
        )
        cv2.line(
            img_left,
            (int(pred_cx), int(pred_cy - 10)),
            (int(pred_cx), int(pred_cy + 10)),
            (int(color[0]), int(color[1]), int(color[2])),
            2,
        )

        # draw the line connecting the center points of the last 10 frames
        # for i in range(1, 10):
        #     if frame - i < 0:
        #         break
        #     df_frame_prev = df[df["frame"] == frame - i]
        #     row_prev = df_frame_prev[df_frame_prev["object_id"] == object_id]
        #     if len(row_prev) == 0:
        #         break
        #     pred_x_prev = row_prev["prediciton_x"].values[0]
        #     pred_y_prev = row_prev["prediciton_y"].values[0]
        #     cv2.line(
        #         img_left,
        #         (int(pred_x), int(pred_y)),
        #         (int(pred_x_prev), int(pred_y_prev)),
        #         (int(color[0]), int(color[1]), int(color[2])),
        #         2,
        #     )

        # add the overlay to the image
        cv2.addWeighted(overlay, 0.3, img_left, 0.7, 0, img_left)

        # Draw an arrow from the center of the bounding box to the center of the prediction
        cv2.arrowedLine(
            img_left,
            (int(bbox_cx), int(bbox_cy)),
            (int(pred_cx), int(pred_cy)),
            (int(color[0]), int(color[1]), int(color[2])),
            2,
        )

        

    # save the image
    cv2.imwrite(
        f"{data_dir}/results/frame_{frame}.jpg", img_left
    )
    video_writer.write(img_left)


    

Processing frame 0: 0000000000.png
Processing frame 1: 0000000001.png
Processing frame 2: 0000000002.png
Processing frame 3: 0000000003.png
Processing frame 4: 0000000004.png
Processing frame 5: 0000000005.png
Processing frame 6: 0000000006.png
Processing frame 7: 0000000007.png
Processing frame 8: 0000000008.png
Processing frame 9: 0000000009.png
Processing frame 10: 0000000010.png
Processing frame 11: 0000000011.png
Processing frame 12: 0000000012.png
Processing frame 13: 0000000013.png
Processing frame 14: 0000000014.png
Processing frame 15: 0000000015.png
Processing frame 16: 0000000016.png
Processing frame 17: 0000000017.png
Processing frame 18: 0000000018.png
Processing frame 19: 0000000019.png
Processing frame 20: 0000000020.png
Processing frame 21: 0000000021.png
Processing frame 22: 0000000022.png
Processing frame 23: 0000000023.png
Processing frame 24: 0000000024.png
Processing frame 25: 0000000025.png
Processing frame 26: 0000000026.png
Processing frame 27: 0000000027.png
Pr

In [12]:
x = np.array([[0],
              [0],
              [0],
              [0],
              [0],
              [0],
              [0],
              [0],
              [0]])

In [13]:
np.dot

<function numpy.dot>